In [27]:
import pandas as pd
import torch
from torch.autograd import Variable
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import random

In [28]:

tracknames = os.listdir("/home/jasper-ubuntu/Documents/Studie/Master AI/Computational intelligence/Torcs-CI/CI/train_data/")
# tracknames = ['aalborg.csv']
datadictin = {}
datadictout = {}
datadict = {}
framesin =[]
framesout=[]
data = []

for i, file in  enumerate(tracknames):
    path = os.path.join('train_data', file)
    print(path)
    datadictin[i] = pd.read_csv(path,sep=';', index_col=False).iloc[:-1, 3:]
    datadictout[i] = pd.read_csv(path,sep=';', index_col=False).iloc[:-1, 0:3]
    datadict[i]=pd.read_csv(path,sep=';', index_col=False).iloc[:,:]

#Concatenate all datasets
for i,file in enumerate(tracknames):
    framesin.append(datadictin[i])
    framesout.append(datadictout[i])
    data.append(datadict[i])
    
inp = pd.concat(framesin) 
outp = pd.concat(framesout) 
train = pd.concat(data)

#Drop missing values
inp.replace('', np.nan, inplace=True)
inp.dropna(inplace=True)

outp.replace('', np.nan, inplace=True)
outp.dropna(inplace=True)

train.replace('', np.nan, inplace=True)
train.dropna(inplace=True)
train.head()

train_data/road_alpine-1_7cars_2475231.csv
train_data/road_alpine-1_7cars_2475119.csv
train_data/road_alpine-1_7cars_2481619.csv
train_data/road_alpine-1_7cars_2481544.csv
train_data/road_alpine-1_7cars_2475043.csv
train_data/road_alpine-1_7cars_2475155.csv
train_data/road_alpine-1_7cars_2481731.csv
train_data/road_alpine-1_7cars_2481655.csv
train_data/road_alpine-1_7cars_2481432.csv
train_data/road_alpine-1_7cars_2481842.csv


,accel,brake,steer,angle,curLapTime,distFromStart,distRaced,gear,lastLapTime,racePos,...,oppos26,oppos27,oppos28,oppos29,oppos30,oppos31,oppos32,oppos33,oppos34,oppos35
0,1.0,0.0,0.0,3.019920e-07,-0.982,6306.65,0.0,0,0.0,7,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
1,1.0,0.0,0.0,3.019920e-07,-0.962,6306.65,0.0,0,0.0,7,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
2,1.0,0.0,0.0,3.019920e-07,-0.942,6306.65,0.0,0,0.0,7,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
3,1.0,0.0,0.0,3.019920e-07,-0.922,6306.65,0.0,0,0.0,7,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
4,1.0,0.0,0.0,3.019920e-07,-0.902,6306.65,0.0,0,0.0,7,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0


In [29]:
input_size = inp.shape[1]
output_size = outp.shape[1]
datapoints = inp.shape[0]
print(datapoints,input_size,output_size)

331770 72 3


In [30]:
# Hyper Parameters
input_size = inp.shape[1]
hidden_size = 50
output_size = outp.shape[1]
num_epochs = 20
batch_size = 100

In [31]:

# Neural Network Model
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h = F.tanh(self.fc1(x))
        h = F.tanh(self.fc2(h))
        h = F.tanh(self.fc3(h))
        return h

In [32]:
# def minibatch(data, batch_size=32):
#     for i in range(0, len(data), batch_size):
#         yield data[i:i+batch_size]

In [33]:
def get_variable(data, labels = False):
    """Get a Variable given data (pandas dataframe)"""
    # Use float (not double) for better precission: (see reply of smth)
    # https://discuss.pytorch.org/t/problems-with-target-arrays-of-int-int32-types-in-loss-functions/140/3
    
    # Labels need to be long to work (for cross entropy, for MSE they both need to be long)
    if labels:
        tensor = torch.from_numpy(data.iloc[:,:].as_matrix()).float()
    else:
        tensor = torch.from_numpy(data.iloc[:,:].as_matrix()).float()
#     print(tensor)
    return Variable(tensor)

In [34]:
def preprocess(batch):
    """ ... """

    x = batch.iloc[:-1, 3:]
    y = batch.iloc[:-1, 0:3]

    return x, y

In [35]:
model = Net(input_size, hidden_size, output_size)

In [36]:
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [37]:
for ITER in range(num_epochs):
    
    # Take a random sample of size batchsize
    
    train_loss = 0.0
    start = time.time()
    updates = 0

    for i in range(int(datapoints/(batch_size))):
        batch = train.sample(n = batch_size)
        updates += 1

        # pad data with zeros
        x, y = preprocess(batch)
        
        # forward pass
        scores = model(get_variable(x))
        targets = get_variable(y, labels= True)
        loss = nn.MSELoss()
        output = loss(scores, targets)
        train_loss += output.data[0]

        # backward pass
        model.zero_grad()
        output.backward()

        # update weights
        optimizer.step()

    print("iter %r: avg train loss=%.4f, time=%.2fs" %
          (ITER, train_loss/updates, time.time()-start))

    # evaluate
#     _, _, acc_train = evaluate(model, train)
#     _, _, acc_dev = evaluate(model, dev)
#     print("iter %r: train acc=%.4f  test acc=%.4f" % (ITER, acc_train, acc_dev))

iter 0: avg train loss=0.0530, time=41.49s
iter 1: avg train loss=0.0521, time=43.74s
iter 2: avg train loss=0.0518, time=40.25s
iter 3: avg train loss=0.0526, time=37.70s
iter 4: avg train loss=0.0530, time=37.45s
iter 5: avg train loss=0.0525, time=55.67s
iter 6: avg train loss=0.0532, time=44.90s
iter 7: avg train loss=0.0528, time=43.10s
iter 8: avg train loss=0.0526, time=38.77s
iter 9: avg train loss=0.0528, time=38.77s
iter 10: avg train loss=0.0526, time=39.77s
iter 11: avg train loss=0.0528, time=40.91s
iter 12: avg train loss=0.0526, time=41.27s
iter 13: avg train loss=0.0532, time=39.13s
iter 14: avg train loss=0.0529, time=38.70s
iter 15: avg train loss=0.0528, time=38.35s
iter 16: avg train loss=0.0530, time=40.31s
iter 17: avg train loss=0.0529, time=55.48s
iter 18: avg train loss=0.0528, time=64.30s
iter 19: avg train loss=0.0528, time=66.90s


In [38]:
# Test prediction
testval = get_variable(x)
print(testval)
print(model(testval))
print(y)

Variable containing:
     0.0009     10.3680    561.8210  ...     200.0000    200.0000    200.0000
    -0.1226      9.3880    517.5990  ...     200.0000    200.0000    200.0000
     0.0871     16.2580    613.1340  ...     200.0000    200.0000    200.0000
                ...                   ⋱                   ...                
     0.0906     66.5700   3094.0701  ...     200.0000    200.0000    200.0000
     0.0636     87.9220   3543.2000  ...     200.0000    200.0000    200.0000
     0.4401     50.6260   2395.4299  ...     200.0000    200.0000    200.0000
[torch.FloatTensor of size 99x72]

Variable containing:
 0.8558  0.0140 -0.0034
 0.8558  0.0140 -0.0034
 0.8558  0.0140 -0.0034
 0.8558  0.0140 -0.0034
 0.8558  0.0140 -0.0034
 0.8558  0.0140 -0.0034
 0.8558  0.0140 -0.0034
 0.8234  0.0362  0.0117
 0.8558  0.0140 -0.0034
 0.8558  0.0140 -0.0034
 0.8558  0.0140 -0.0034
 0.8558  0.0140 -0.0034
 0.8558  0.0140 -0.0034
 0.8234  0.0362  0.0117
 0.8558  0.0140 -0.0034
 0.8234  0.0362  

In [40]:
save_path_model = 'simple_ff_model_20epochs_batch100.pt'
save_path_dict = 'simple_ff_DICT_20epochs_batch100_dict_hiden_size50_2HL_INP72_outp3_Forward_tanh_tanh_tanh.pt'
# https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
torch.save(model, save_path_model)
torch.save(model.state_dict(),save_path_dict)
# model.save_state_dict('trainedmodel

/home/jasper-ubuntu/miniconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
